# Export des tokens sous format CSV depuis un XML révisé manuellement

In [16]:
#Revue le 15 avril. Options pour fichier déjà lemmatisé ajoutées.

def export_tokens_to_csv(chemin_entree, chemin_sortie):
    
    """
    Fonction récupérant les éléments <w> d'un fichier XML-TEI
    avec leurs @n pour les exporter dans un fichier CSV à
    deux colonnes, l'une pour le numéro, l'autre pour le texte.
    Si l'élément <w> comporte des enfants <height> pour les initiales
    hautes, ou <choice> pour des modernisations/abréviations,
    la fonction compilera une version complète, résolue et modernisée
    pour le logiiel Analog.
    
    :param chemin_entree: Le chemin local vers un fichier en XML-TEI
        dont on souhaite extraire les tokens pour Analog.
    :param chemin_sortie: L'emplacement local dans lequel on souhaite
        écrire le fichier CSV à donner à Analog.    
    """

    import xml.etree.ElementTree as ET
    import csv
    
    # Le dictionnaire dans lequel seront stockés les token avant écriture en CSV.
    # dico_tokens = {}
    # Les colonnes du futur fichier CSV.
    # colonnes = ['ID', 'TOKEN']
    colonnes = ['ID', 'TOKEN', 'LEMMES', 'POS']
    
    # Pour que Python comprenne les éléments dont on parlera,
    # il faut lui donner la déclaration TEI, mais comme c'est
    # la seule qu'on utilisera, pas besoin de lui donner un préfixe.
    # ET.register_namespace('', "http://tei-c.org/ns/1.0")
    
    # On va chercher le fichier XML-TEI et on le lit.
    tree = ET.parse(chemin_entree)
    root = tree.getroot()
    
    # On ouvre le CSV de sortie en mode "écriture", on y écrit le nom des colonnes.
    with open(chemin_sortie, 'w') as csv_file:
        csv_contenu = csv.DictWriter(csv_file, fieldnames = colonnes, delimiter=";")
        csv_contenu.writeheader()
        
        # On boucle sur les éléments <w> du XML, dans l'ordre du fichier.
        # for word in root.findall('.//{http://tei-c.org/ns/1.0}w'):
        for word in root.findall('.//w'):
        
            # On récupère les @n, @lemma et @pos dans les variables
            # "numero", "lemmes" et "pos"
            # et on crée la chaîne "texte", pour l'instant vide.
            numero = str(word.get('n'))
            lemmes = str(word.get('lemma'))
            pos = str(word.get('pos'))
            texte = ""
        
            # Si <w> n'a pas d'enfant, on récupère le texte tel-quel.
            if word.find('.') == None :
                texte = str(word.text)
            
            # Sinon, on compile.
            else:
            
                # S'il y a du texte avant le premier enfant, on l'ajoute.
                if word.text:
                    texte += str(word.text)
                
                # On boucle sur les enfants du <w> actuel.
                for item in word:
                
                    # Si l'enfant est un <height>, on récupère son texte.
                    if item.tag == 'height' or item.tag == 'supplied':
                        texte += str(item.text)
                        # S'il y a du texte après la balise fermante et avant
                        # le prochain enfant ou la balise fermante du <w>,
                        # on l'ajoute.
                        if item.tail:
                            texte += str(item.tail)
                        
                    elif item.tag == 'lb':
                        if item.tail:
                            texte += str(item.tail)
                        
                    # Si l'enfant est un <choice>, on récupère le texte de son
                    # second enfant et on vérifie s'il y a du texte après le <choice>.
                    elif item.tag == 'choice':
                        texte += str(item[1].text)
                        if item.tail:
                            texte += str(item.tail)
                            
                    elif item.tag == 'c':
                        texte += item.text
                        if item.tail:
                            texte += str(item.tail)
                            
                    elif item.tag == 'hi':
                        texte += item.text
                        if item.tail:
                            texte += item.tail
                            
                    elif item.tag == 'add':
                        # On refait tous les tests.
                        if item.find('.') == None :
                            texte = str(item.text)
                            
                        else:
                            
                            if item.text:
                                texte += str(item.text)
                            
                            for subitem in item:
                                if subitem.tag == 'lb':
                                    if subitem.tail:
                                        texte += str(subitem.tail)
                                elif subitem.tag == 'choice':
                                    texte += str(subitem[1].text)
                                    if subitem.tail:
                                        texte += str(subitem.tail)
                
                
                
            # Une fois le mot copié ou compilé, on ajoute une unité au dictionnaire de tokens,
            # dont la clé est le numéro (le @) et la valeur est la chaîne copiée ou compilée.
            # dico_tokens[str(numero)] = str(texte)
    
        # Puis on écrit les valeurs du mot dans les colonnes appropriées.
    
            csv_contenu.writerow(
                {
                    "ID" : numero,
                    "TOKEN" : str(texte),
                    "LEMMES" : lemmes,
                    "POS" : pos
                }
            )

In [17]:
export_tokens_to_csv('/home/erminea/Documents/CONDE/editions/base-version/rouille_base.xml',
                 '/home/erminea/Documents/CONDE/Encodage/ROUILLE/rouille_tokens_decalles.csv')

In [2]:
# Pour exécuter la fonction, remplacer les chemins par les chemins actuels.

export_tokens_to_csv('/home/erminea/Documents/CONDE/Encodage/basnage_renum.xml',
                 '/home/erminea/Documents/CONDE/Encodage/basnage_tokens.csv')

export_tokens_to_csv('/home/erminea/Documents/CONDE/Encodage/berault_renum.xml',
                 '/home/erminea/Documents/CONDE/Encodage/berault_tokens.csv')

export_tokens_to_csv('/home/erminea/Documents/CONDE/Encodage/instructions_renum.xml',
                 '/home/erminea/Documents/CONDE/Encodage/instructions_tokens.csv')

export_tokens_to_csv('/home/erminea/Documents/CONDE/Encodage/merville_renum.xml',
                 '/home/erminea/Documents/CONDE/Encodage/merville_tokens.csv')

export_tokens_to_csv('/home/erminea/Documents/CONDE/Encodage/pesnelle_renum.xml',
                 '/home/erminea/Documents/CONDE/Encodage/pesnelle_tokens.csv')

export_tokens_to_csv('/home/erminea/Documents/CONDE/Encodage/rouille_renum.xml',
                 '/home/erminea/Documents/CONDE/Encodage/rouille_tokens.csv')

export_tokens_to_csv('/home/erminea/Documents/CONDE/Encodage/ruines_renum.xml',
                 '/home/erminea/Documents/CONDE/Encodage/ruines_tokens.csv')

export_tokens_to_csv('/home/erminea/Documents/CONDE/Encodage/tac_renum.xml',
                 '/home/erminea/Documents/CONDE/Encodage/tac_tokens.csv')

export_tokens_to_csv('/home/erminea/Documents/CONDE/Encodage/terrien_renum.xml',
                 '/home/erminea/Documents/CONDE/Encodage/terrien_tokens.csv')